In [1]:
import geopandas as gpd

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [1]:
import pandas as pd

In [2]:
path = r'G:\projects\UtilityDistricts\eweb\DrinkingWater\IllegalCampCoordination\Recieved\IllegalCampNotification_pro'

In [3]:
dat = pd.read_excel(path+'\\most_recent.xlsx')

In [5]:
dat.columns

Index(['OBJECTID', 'Join_Count', 'TARGET_FID', 'Status', 'Comments', 'Date',
       'Submitted_by', 'Dogs_present', 'Unruly_inhabitants',
       'Hazardous_materials_present', 'Biohazards_present',
       'Size_of_encampment', 'maptaxlot_hyphen', 'ownname', 'addr1',
       'ownercity', 'ownerprvst', 'ownerzip', 'geocity_name', 'ugb_name',
       'longitude', 'latitude'],
      dtype='object')

In [ ]:
cols2drop = ['OBJECTID', 'Join_Count',]

In [4]:
dat = gpd.read_file(path + '\\MyProject4.gdb', driver='FileGDB', layer='datacopy')

In [5]:
dat.head(3)

,Status,Comments,Date,Submitted_by,Dogs_present,Unruly_inhabitants,Hazardous_materials_present,Biohazards_present,Size_of_encampment,Date_str,geometry
0,Abandoned,Contcated group 5/18 and gave notice.,2016-05-18T15:36:59+00:00,Youngblood,No,No,None,None,Small,5/18/2016,POINT (648676.895 837520.043)
1,Cleaned-up,Cleaned up 5/18.,2016-05-18T15:38:19+00:00,Youngblood,No,No,No,No,Small,5/18/2016,POINT (648704.972 837641.883)
2,Occupied,Notified campers 5/18.,2016-05-18T15:39:57+00:00,Youngblood,No,None,None,None,Small,5/18/2016,POINT (648423.293 837925.831)


In [10]:
type(dat.Date.values[0])

str